In [15]:
import cv2
import numpy as np

def fog_image(clean_img, A=0.5, i=0, alpha=0.004):
    """
    Generate a foggy image from a clean image using:
        d(x) = alpha * rho(x),
        t(x) = exp(-beta * d(x)),
        I(x) = J(x)*t(x) + A*(1 - t(x)),

    where:
        rho(x) is the distance from the image center.
        beta = 0.05 * i      <-- so beta=0 when i=0
        alpha is a user-chosen scaling factor for the distance.

    Parameters
    ----------
    clean_img : np.ndarray (H,W,C)
    A : float
        Global atmospheric light, e.g. 0.5
    i : int
        Fog level index (0 to 9).
    alpha : float
        Scales distance -> depth. Adjust this to get stronger/weaker fog.

    Returns
    -------
    foggy_img : np.ndarray (H,W,C) in uint8
    """
    # Convert to float32 in [0, 1]
    if clean_img.dtype != np.float32:
        J = clean_img.astype(np.float32) / 255.0
    else:
        J = clean_img
    
    rows, cols = J.shape[:2]
    
    # Create meshgrid for pixel coordinates
    yy, xx = np.mgrid[0:rows, 0:cols]
    
    # Distance of each pixel from image center
    cy, cx = rows / 2, cols / 2
    rho = np.sqrt((yy - cy)**2 + (xx - cx)**2)
    
    # Ensure i=0 => beta=0, so no fog
    beta = 0.05 * i   # i in [0..9], so range is [0..0.45]
    
    # Depth map: d(x) = alpha * rho(x)
    d = alpha * rho
    
    # Transmission map: t(x) = exp(-beta * d)
    t = np.exp(-beta * d)
    
    # Reshape t to broadcast over color channels
    if len(J.shape) == 3 and J.shape[2] == 3:
        t_3d = np.repeat(t[..., np.newaxis], 3, axis=2)
    else:
        t_3d = t  # for grayscale
    
    # I(x) = J(x)*t + A*(1 - t)
    foggy_img = J * t_3d + A * (1.0 - t_3d)
    
    # Convert back to uint8
    foggy_img = np.clip(foggy_img * 255.0, 0, 255).astype(np.uint8)
    return foggy_img

def main():
    # 1. Load a clean image
    clean_img = cv2.imread("/home/hawk/Desktop/CMU/tofoggy.jpg")
    if clean_img is None:
        raise ValueError("Could not open or find the image. Check the file path.")
    
    # 2. Generate and save 10 foggy images (i from 0 to 9)
    for i in range(10):
        foggy = fog_image(clean_img, A=0.5, i=i, alpha=0.004)
        out_name = f"fog_level_{i}.png"
        cv2.imwrite(out_name, foggy)
        print(f"Saved {out_name}")

if __name__ == '__main__':
    main()


Saved fog_level_0.png
Saved fog_level_1.png
Saved fog_level_2.png
Saved fog_level_3.png
Saved fog_level_4.png
Saved fog_level_5.png
Saved fog_level_6.png
Saved fog_level_7.png
Saved fog_level_8.png
Saved fog_level_9.png


In [18]:
import cv2
import numpy as np

def darken_image(input_img, darkening_factor=0.9):
    """
    Darken the image by a linear factor (0 < darkening_factor < 1).
    You could replace this with a gamma correction if you prefer.
    """
    # Convert to float in [0,1]
    img_float = input_img.astype(np.float32) / 255.0
    # Apply darkening
    darkened = img_float * darkening_factor
    # Convert back to uint8
    out = np.clip(darkened * 255, 0, 255).astype(np.uint8)
    return out

def light_fog_attenuation(input_img, visibility_distance=500, atmospheric_light=0.8):
    """
    Apply a mild fog effect with high visibility (meaning it won't be very strong).
    We use a simple center-based depth approach. 
    For real fog, you might have a true depth map. Here, we fake it.
    """
    # Convert to float in [0,1]
    img_float = input_img.astype(np.float32) / 255.0
    h, w = img_float.shape[:2]

    # Create a 'fake' depth based on distance from center
    yy, xx = np.mgrid[0:h, 0:w]
    cy, cx = h / 2, w / 2
    rho = np.sqrt((yy - cy)**2 + (xx - cx)**2)  # distance from center

    # We'll define beta from the 'visibility_distance'
    # Typically, visibility ~ 1/beta in very rough approximations, 
    # so let's do beta = 1 / visibility_distance
    beta = 1.0 / max(visibility_distance*2, 1e-6)

    # Transmission map: t(x) = exp(-beta * depth). We'll treat depth ~ rho.
    # We scale 'rho' so that corners have ~ some max depth.
    # This is just a toy approach.
    rho_scaled = rho / (max(h, w) * 0.5)  # normalized 0..1 near corners
    t = np.exp(-beta * rho_scaled * max(h, w))  # rough scaling

    # Reshape for broadcast if 3 channels
    if len(img_float.shape) == 3 and img_float.shape[2] == 3:
        t_3d = np.repeat(t[..., np.newaxis], 3, axis=2)
    else:
        t_3d = t

    # Fog formula: Ifog(x) = I(x)*t(x) + A*(1 - t(x))
    foggy = img_float * t_3d + atmospheric_light * (1.0 - t_3d)

    # Convert back to uint8
    out = np.clip(foggy * 255, 0, 255).astype(np.uint8)
    return out

def generate_rain_layer(shape,
                        gauss_mean=0,
                        gauss_std=0.3,
                        motion_blur_ksize=15,
                        motion_angle=-25,
                        scale=1.0,
                        brightness_scale=1.5):
    """
    Create a 'rain streak' layer by:
      1) generating 2D Gaussian noise
      2) applying motion blur
      3) optional scale/crop
      4) color/level adjustment

    Returns a float32 [H,W,3] with prominent white streaks on black.
    """

    h, w = shape[:2]

    # 1) Gaussian noise in 2D
    noise = np.random.normal(gauss_mean, gauss_std, (h, w)).astype(np.float32)

    # 2) Motion blur
    # Create a 'vertical line' kernel and rotate it by motion_angle
    kernel = np.zeros((motion_blur_ksize, motion_blur_ksize), dtype=np.float32)
    kernel[:, motion_blur_ksize // 2] = 1.0
    kernel /= motion_blur_ksize

    center = (motion_blur_ksize / 2, motion_blur_ksize / 2)
    rot_mat = cv2.getRotationMatrix2D(center, motion_angle, 1.0)
    kernel_rotated = cv2.warpAffine(kernel, rot_mat, (motion_blur_ksize, motion_blur_ksize))

    streaks = cv2.filter2D(noise, -1, kernel_rotated)

    # 3) Scale if needed (simulate near/far streaks). 
    #    Here, we do a uniform scale, but you could do random crops to vary.
    if abs(scale - 1.0) > 1e-3:
        new_w = int(w * scale)
        new_h = int(h * scale)
        # Resize up or down
        streaks_resized = cv2.resize(streaks, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        # Then crop (or pad) back to original (h, w)
        if new_h >= h and new_w >= w:
            # crop center
            start_y = (new_h - h) // 2
            start_x = (new_w - w) // 2
            streaks = streaks_resized[start_y:start_y+h, start_x:start_x+w]
        else:
            # If smaller, center it on black
            out = np.zeros((h, w), dtype=np.float32)
            y_off = (h - new_h) // 2
            x_off = (w - new_w) // 2
            out[y_off:y_off+new_h, x_off:x_off+new_w] = streaks_resized
            streaks = out

    # 4) Color/level adjustment to keep "prominent" streaks
    #    We'll just do a simple brightness scaling and clamp to [0,1].
    #    Then interpret negative values as black, positive as white streaks.
    streaks *= brightness_scale
    streaks = np.clip(streaks, 0.0, 1.0)  # keep in [0..1]

    # Convert to a 3-channel image (white streaks on black)
    streaks_3c = np.dstack([streaks, streaks, streaks])  # shape: [H,W,3], float32 in [0..1]

    return streaks_3c

def blend_rain_layer(base_img, rain_layer, alpha=0.3):
    """
    Blend the rain layer with the base image using:
      Iblend = Io*(1-alpha) + l*alpha
    base_img, rain_layer assumed float32 [0..1].
    alpha in [0..1].
    """
    return base_img * (1 - alpha) + rain_layer * alpha

def rain_augmentation_pipeline(input_img,
                              darkening_factor=0.9,
                              visibility_distance=500,
                              atmospheric_light=0.8,
                              gauss_std=0.3,
                              motion_blur_ksize=15,
                              motion_angle=-25,
                              scale=1.0,
                              brightness_scale=1.5,
                              alpha=0.3):
    """
    An example pipeline combining:
      1) Darkening
      2) Light fog attenuation
      3) Rain layer generation
      4) Final blend
    """

    # 1) Darken
    darkened = darken_image(input_img, darkening_factor=darkening_factor)

    # 2) Light Fog
    fogged = light_fog_attenuation(darkened, 
                                   visibility_distance=visibility_distance,
                                   atmospheric_light=atmospheric_light)

    # Convert to float in [0,1] for further blending
    base_float = fogged.astype(np.float32) / 255.0

    # 3) Generate rain streak layer
    rain_layer = generate_rain_layer(shape=fogged.shape,
                                     gauss_std=gauss_std,
                                     motion_blur_ksize=motion_blur_ksize,
                                     motion_angle=motion_angle,
                                     scale=scale,
                                     brightness_scale=brightness_scale)

    # 4) Final blend: Iblend = Io*(1-alpha) + l*alpha
    #    Where Io = 'base_float' and l = 'rain_layer'.
    out_float = blend_rain_layer(base_float, rain_layer, alpha=alpha)

    # Convert back to uint8
    out_img = np.clip(out_float * 255.0, 0, 255).astype(np.uint8)
    return out_img

def main():
    # Example usage
    clean_img = cv2.imread("/home/hawk/Desktop/CMU/tofoggy.jpg")
    if clean_img is None:
        raise IOError("Could not open or find 'clean_image.jpg'.")

    # Try default parameters (light fog + mild rain)
    out_rain = rain_augmentation_pipeline(
        clean_img,
        darkening_factor=0.85,     # make image a bit darker
        visibility_distance=600,   # large => very mild fog
        atmospheric_light=0.8,     # light color for the mild fog
        gauss_std=0.3,            # standard deviation for noise
        motion_blur_ksize=15,     # size for motion blur kernel
        motion_angle=-30,         # tilt for rain streaks
        scale=1.0,                # scale for streak size
        brightness_scale=1.8,     # how bright the streaks are
        alpha=0.3                 # how strongly to blend the streaks
    )

    cv2.imwrite("rain_augmented.jpg", out_rain)
    print("Saved 'rain_augmented.jpg' with synthetic rain.")

if __name__ == "__main__":
    main()


Saved 'rain_augmented.jpg' with synthetic rain.
